In [29]:
import pandas as pd
following = pd.read_csv("following.csv")
profiles = pd.read_csv("data/sampled_profiles.csv")

In [30]:
def get_did(profiles, handle):
    return str(profiles[profiles['handle'] == handle]['did'].values[0])

print(get_did(profiles, 'lukelukeluke.bsky.social'))

did:plc:eg336dt7kyike5xkyed3iwcv


In [36]:
def get_user_following_the_same_account(df, user_did):
    # Filter DataFrame to get rows with the same subject as user_did
    same_subject_df = df[df['subject'].isin(df[df['did'] == user_did]['subject'])]
    
    # Filter DataFrame to get rows with different did than user_did
    different_did_df = same_subject_df[same_subject_df['did'] != user_did]
    
    # Get unique did values
    unique_dids = different_did_df['did'].unique()
    
    # Calculate the number of overlapping subjects for each DID
    overlapping_counts = []
    for did in unique_dids:
        overlapping_count = same_subject_df[same_subject_df['did'] == did]['subject'].nunique()
        overlapping_counts.append((did, overlapping_count))
    
    # Create a DataFrame to store the overlapping counts
    overlapping_df = pd.DataFrame(overlapping_counts, columns=['did', 'overlapping_subject_count'])
    
    # Sort the DataFrame by overlapping subject count in descending order
    overlapping_df = overlapping_df.sort_values(by='overlapping_subject_count', ascending=False)
    
    return overlapping_df

get_user_following_the_same_account(following, "did:plc:snjxvu5fmqeyto5dqlwxjwgq")

did = get_did(profiles, handle = input("input your handle:"))
filtered = get_user_following_the_same_account(following, did)
filtered['user_name'] = profiles['handle']
filtered.drop('did', axis=1, inplace=True)
total = profiles[profiles['did'] == did]['follows_count']

# Check if filtered and total DataFrames are not empty before accessing their columns
if not filtered.empty and not total.empty:
    filtered['overlapping_subject_count'] = filtered['overlapping_subject_count'] / total.iloc[0]
    print(filtered)
else:
    print("Filtered or total DataFrame is empty.")

IndexError: index 0 is out of bounds for axis 0 with size 0